In [ ]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math 
import copy

'''
Model code from
- https://medium.com/towards-data-science/build-your-own-transformer-from-scratch-using-pytorch-84c850470dcb

'''


'''Simple Multihead Attention'''
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        print("Scaled dot product attention")
        print("Q shape: ", Q.shape)
        print("K shape: ", K.shape)
        print("V shape: ", V.shape)
        print("Mask shape: ", mask.shape)
        print()
        
        ########
        ########
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        print("Attention scores shape: ", attn_scores.shape)
        
        
        attn_probs = torch.softmax(attn_scores, dim=-1)
        print("Attention probs shape: ", attn_probs.shape)
        
        
        output = torch.matmul(attn_probs, V)
        print("Output attention shape: ", output.shape)
        print()
        ######
        ######
        
        
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        print("Before Splitting")
        print("Q shape: ", Q.shape)
        print("K shape: ", K.shape)
        print("V shape: ", V.shape)
        print("Mask shape: ", mask.shape)
        print()
        
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        print("After Splitting")
        print("Q shape: ", Q.shape)
        print("K shape: ", K.shape)
        print("V shape: ", V.shape)
        print()
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        print("Attention output shape: ", attn_output.shape)
        
        output = self.W_o(self.combine_heads(attn_output))
        return output
    
'''Position-wise Feed-Forward Networks'''
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))
    
'''Positional Encoding'''
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]
    
'''Encoder Layer'''
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        print("X for Q, K, V: ", x.shape)
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x
    
'''Decoder Layer'''
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x
    
    
'''Transformer Model'''
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        

        
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        print("Generate mask")
        print("src shape: ", src.shape)
        print("tgt shape: ", tgt.shape)
        print("src mask shape: ", src_mask.shape)
        print("tgt mask shape: ", tgt_mask.shape)
        print()
        
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))
        
        print("src embedded shape: ", src_embedded.shape)
        print("tgt embedded shape: ", tgt_embedded.shape)
        print()

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output
    
    
'''Test'''
# Data
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)


# Training
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(1):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()

    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Generate mask
src shape:  torch.Size([64, 100])
tgt shape:  torch.Size([64, 99])
src mask shape:  torch.Size([64, 1, 1, 100])
tgt mask shape:  torch.Size([64, 1, 99, 99])

src embedded shape:  torch.Size([64, 100, 512])
tgt embedded shape:  torch.Size([64, 99, 512])

X for Q, K, V:  torch.Size([64, 100, 512])
Before Splitting
Q shape:  torch.Size([64, 100, 512])
K shape:  torch.Size([64, 100, 512])
V shape:  torch.Size([64, 100, 512])
Mask shape:  torch.Size([64, 1, 1, 100])

After Splitting
Q shape:  torch.Size([64, 8, 100, 64])
K shape:  torch.Size([64, 8, 100, 64])
V shape:  torch.Size([64, 8, 100, 64])

Scaled dot product attention
Q shape:  torch.Size([64, 8, 100, 64])
K shape:  torch.Size([64, 8, 100, 64])
V shape:  torch.Size([64, 8, 100, 64])
Mask shape:  torch.Size([64, 1, 1, 100])

Attention scores shape:  torch.Size([64, 8, 100, 100])
Attention probs shape:  torch.Size([64, 8, 100, 100])
Output attention shape:  torch.Size([64, 8, 100, 64])

Attention output shape:  torch.Si

In [10]:
## TESTING SHAPE of Data
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)



In [12]:
print(src_data.shape)
print(tgt_data.shape)

torch.Size([64, 100])
torch.Size([64, 100])


## II. From scratch softmax vs. Pytorch softmax

### 1. Scratch Softmax

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Softmax(nn.Module):
    def __init__(self, dim=None):
        super(Softmax, self).__init__()
        self.dim = dim

    def forward(self, x):
        """
        Computes softmax along the specified dimension.

        Args:
            x (torch.Tensor): Input tensor.
            dim (int, optional): Dimension along which to compute softmax. If None, uses the dimension specified in __init__. Defaults to None.

        Returns:
            torch.Tensor: Softmax output.
        """
        if self.dim is None:
            dim = -1  # Apply softmax over the last dimension if dim is not specified
        else:
            dim = self.dim

        # Subtract the maximum value for numerical stability
        x = x - torch.max(x, dim=dim, keepdim=True)[0]

        # Compute exponentials
        exp_x = torch.exp(x)

        # Compute softmax
        softmax_x = exp_x / torch.sum(exp_x, dim=dim, keepdim=True)

        return softmax_x
    


In [5]:
x = torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.float32)
print(x.shape)
softmax = Softmax(dim=1)

output = softmax(x)
print(output)
print(output.shape)

torch.Size([2, 3])
tensor([[0.0900, 0.2447, 0.6652],
        [0.0900, 0.2447, 0.6652]])
torch.Size([2, 3])


In [7]:
# Torch function 
x = torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.float32)

output = F.softmax(x, dim=1)
print(output)
output = torch.softmax(x, dim=1)
print(output)



tensor([[0.0900, 0.2447, 0.6652],
        [0.0900, 0.2447, 0.6652]])
tensor([[0.0900, 0.2447, 0.6652],
        [0.0900, 0.2447, 0.6652]])


In [9]:
## Bigger Tensor Example
ex_tensor = torch.randn(3, 28, 28)
print(ex_tensor.shape)

# Softmax along the last dimension
softmax = Softmax(dim=-1)
output = softmax(ex_tensor)
print(output.shape)
print(output)

torch.Size([3, 28, 28])
torch.Size([3, 28, 28])
tensor([[[0.0213, 0.0178, 0.0111,  ..., 0.0076, 0.0262, 0.0122],
         [0.0137, 0.0319, 0.0183,  ..., 0.0050, 0.0087, 0.0148],
         [0.0169, 0.0396, 0.0044,  ..., 0.0102, 0.1815, 0.0159],
         ...,
         [0.0349, 0.0095, 0.0029,  ..., 0.0449, 0.0386, 0.0061],
         [0.0155, 0.0067, 0.0719,  ..., 0.0077, 0.0070, 0.0213],
         [0.0109, 0.0195, 0.0221,  ..., 0.1083, 0.0324, 0.0330]],

        [[0.0021, 0.0063, 0.2122,  ..., 0.0092, 0.0134, 0.0375],
         [0.0925, 0.0058, 0.0484,  ..., 0.0046, 0.0140, 0.0495],
         [0.0290, 0.0459, 0.0523,  ..., 0.0228, 0.0203, 0.0172],
         ...,
         [0.0097, 0.0095, 0.0043,  ..., 0.0087, 0.0308, 0.0069],
         [0.1249, 0.0194, 0.0120,  ..., 0.0255, 0.0868, 0.0467],
         [0.0115, 0.0097, 0.0055,  ..., 0.0105, 0.1680, 0.0097]],

        [[0.1932, 0.0058, 0.0651,  ..., 0.0545, 0.1319, 0.0724],
         [0.0305, 0.0222, 0.0553,  ..., 0.0044, 0.0119, 0.0145],
         [